In [20]:
!pip install snowflake-snowpark-python
!pip install snowflake-ml-python
#pip install googletrans

### Setup Connection

In [90]:
from snowflake.snowpark import Session
import numpy as np
import pandas as pd
# from googletrans import Translator
from snowflake.snowpark.functions import year, month, dayofmonth, dayofweek, to_varchar
import snowflake.ml.modeling.preprocessing as snowml
from snowflake.snowpark.functions import col,sum as sum_, when, avg, median
from snowflake.snowpark.functions import count, lit, min, max
from snowflake.snowpark.types import IntegerType, FloatType, DoubleType, DecimalType, StringType, DateType, TimestampType, LongType
import matplotlib.pyplot as plt
import seaborn as sns

In [91]:
connection_parameters_2 = {
...    "account": "YEZEPEO-DATASCIENCE",
...    "user": "VNSHARED",
...    "password": "Columbia2024!",
...    "role": "SYSADMIN", # optional
...    "warehouse": "ANALYSIS", # optional
...   "database": "MEIXI_DATA", # optional
...   "schema":"PUBLIC",  # optional
...  }

In [92]:
new_session = Session.builder.configs(connection_parameters_2).create()

#### Join Vespucio and equifax data

In [93]:
# df_consolidated=new_session.table('PUBLIC.TBPAGOS_DOCUMENTOSCOBRO_CONSOLIDADOEXTENDIDO_TS_SHARED')
# df_equifax=new_session.table('PUBLIC.EQUIFAX_SHARED')

In [94]:
# df_consolidated=new_session.sql('select * from PUBLIC.TBPAGOS_DOCUMENTOSCOBRO_CONSOLIDADOEXTENDIDO_TS_SHARED')
# df_equifax=new_session.sql('select * from PUBLIC.EQUIFAX_SHARED')

In [95]:
df_consolidated=new_session.sql('select * from PUBLIC.CONSOLIDATED_COPY')
df_equifax = new_session.sql('select * from PUBLIC.EQUIFAX_COPY')


In [96]:
# Finding Common columns
common_cols = np.intersect1d(df_consolidated.columns, df_equifax.columns)

# Printing common columns
print ("Common Columns:",common_cols)

Common Columns: ['"FechaExtraccion"' 'RUT']


In [97]:
df_con_equifax=df_consolidated.join(df_equifax.select('"RUT"', '"Score Equifax"'), on='"RUT"', how='left').drop(df_equifax['"FechaExtraccion"'])

In [98]:
# Define the new column name for the second column
new_column_name = 'FechaExtraccion'

# Use withColumnRenamed to create a new DataFrame with the renamed column
df_con_equifax_v = df_con_equifax.withColumnRenamed(df_con_equifax.columns[1], new_column_name)

In [99]:
df_con_equifax.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"RUT"       |"NumeroDocumento"  |"NumeroFolio"  |"FechaEmision"  |"FechaVencimiento"  |"FechaRegistro"  |"BusinessPartnerId"  |"CuentaContrato"  |"C

In [100]:
# total_rows = df_con_equifax_v.count()

# # Check each column for missing values by comparing non-null counts to total rows
# missing_value_counts = df_con_equifax_v.select(
#     [(total_rows - count(col(c))).alias(c + "_missing") for c in df_con_equifax_v.columns]
# )

# # Show the result, each column name will have a suffix "_missing" showing the count of missing values
# missing_value_counts.show()

In [113]:
df=df_con_equifax_v
#df.show(100)

In [114]:
df = df.with_column("YEAR_DUE", to_varchar(year(col('"FechaVencimiento"'))))\
                        .with_column("MONTH_DUE", to_varchar(month(col('"FechaVencimiento"'))))\
                        .with_column("DAY_OF_MONTH_DUE", to_varchar(dayofmonth(col('"FechaVencimiento"'))))\
                        .with_column("DAY_OF_WEEK_DUE", to_varchar(dayofweek(col('"FechaVencimiento"'))))\

# df = df.with_column("YEAR_PAID", to_varchar(year(col('"FechaCompensacion"'))))\
#                           .with_column("MONTH_PAID", to_varchar(month(col('"FechaCompensacion"'))))\
#                           .with_column("DAY_OF_MONTH_PAID", to_varchar(dayofmonth(col('"FechaCompensacion"'))))\
#                           .with_column("DAY_OF_WEEK_DUE", to_varchar(dayofweek(col('"FechaCompensacion"'))))\

df = df.with_column("YEAR_INVOICED", to_varchar(year(col('"FechaEmision"'))))\
                        .with_column("MONTH_INVOICED", to_varchar(month(col('"FechaEmision"'))))\
                        .with_column("DAY_OF_MONTH_INVOICED", to_varchar(dayofmonth(col('"FechaEmision"'))))\
                        .with_column("DAY_OF_WEEK_INVOICED", to_varchar(dayofweek(col('"FechaEmision"'))))\

df = df.filter(col("RUT").is_not_null())
df = df.withColumn('Unpaid', when(col('"StatusCompensacion"') == 'Impaga', 1).otherwise(0))

#df = df.filter(col('YEAR_INVOICED')!=2024)


#   aggregated_df = df.group_by(col('"BusinessPartnerId"')).agg(
#     min(col("YEAR_INVOICED")).alias("first_year_joined"),
#     max(col("YEAR_INVOICED")).alias("last_year_present"),
#     (max(col("YEAR_INVOICED")) - min(col("YEAR_INVOICED"))).alias("customer_lifespan"),
#     (sum_(col('Unpaid')) / count(col('Unpaid')) * 100).alias("default_rate_percent"),  # Calculating default rate as a percentage
#     median(col('"ImportePesos"')).alias("median_importe_pesos"),
#     median(col('"DiasCompensacion"')).alias("median_dias_compensacion")
#     # avg(col('"Score Equifax"')).alias("average_score_equifax")
# )

#   print(aggregated_df)


#dropping unique identifiers
columns_to_drop = ['"NumeroDocumento"', '"NumeroFolio"', '"BusinessPartnerId"', '"RUT"','"CuentaContrato"','"StatusCompensacionTmp"',
                  '"FechaRegistro"','"ClaveReconciliacion"','"Cabecera_FechaHoraCreacion"','"Detalle_FechaHoraCreacion"','"Direccion"','"Nombre"','FECHAEXTRACCION','"DocumentoCompensacion"','"PosicionDocumento"',
                  ]
df = df.drop(*columns_to_drop)

#over 70% missing values
columns_to_drop2=['"Cabecera_FechaHoraUltimaModificacion"', '"Detalle_FechaHoraUltimaModificacion"']
df = df.drop(*columns_to_drop2)

#redundant information- TipoOperacion contains the same information
columns_to_drop3=['"OperacionPrincipal"','"OperacionParcial"','"TramoMora"']
df = df.drop(*columns_to_drop3)


df = df.with_column('"MaxCategoriaVehiculo"', col('"MaxCategoriaVehiculo"').astype("string"))
df = df.drop('"StatusCompensacion"')
df = df.drop('"DiasCompensacion"')

schema = df.schema

numerical_columns = []
categorical_columns = []
datetime_columns=[]

# Iterate over the schema and check the data type of each column
for field in schema.fields:
  #print(field.datatype)
  if isinstance(field.datatype, (IntegerType, FloatType, DoubleType, DecimalType, LongType)):
      numerical_columns.append(field.name)
  elif isinstance(field.datatype, StringType):
      categorical_columns.append(field.name)
  elif isinstance(field.datatype, (TimestampType, DateType)):
      datetime_columns.append(field.name)

df.show()
df = df.drop(*datetime_columns)
df = df.drop(*categorical_columns)


# for column in categorical_columns:
#   df = df.with_column(column, col(column).cast("category"))

selected_columns = ['"ImportePesos"', '"Score Equifax"']
numerical_columns_x = [col for col in numerical_columns if col in selected_columns]

#categorical_columns_ohe = [(col.strip('"') + '_OHE') for col in categorical_columns]


num_columns_nor = [col + "_NORM" for col in numerical_columns_x]
# print(categorical_columns_ohe)

#Encode categoricals to numeric columns
# snowml_ohe = snowml.OneHotEncoder(input_cols=categorical_columns, output_cols=categorical_columns_ohe)
# transformed_df = snowml_ohe.fit(df).transform(df)

# model = snowml_ohe.fit(df)

# Now you can use this model to transform the same DataFrame
# transformed_df = model.transform(df)
# print(transformed_df.show())

#Scale numerical columns to prevent bias
snowml_mms = snowml.MinMaxScaler(input_cols=numerical_columns_x, output_cols=num_columns_nor)
normalized_df = snowml_mms.fit(df).transform(df)

normalized_df=  normalized_df.drop(*selected_columns)

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"FechaEmision"  |"FechaVencimiento"  |"ImportePesos"  |"FechaCompensacion"  |"COMUNA"   |"CIUDAD"   |"REGION"  |"CantidadCuentasContrato"  |"CantidadVehiculos"  |"TramoMoraCliente"  |"Inhabilitado"  |"Concesionaria"  |"TipoCliente"  |"TipoOperacion"  |"TipoDocumento"      |"TipoProducto"  |"Score Equifax"  |"YEAR_DUE"  |"MONTH_DUE"  |"DAY_OF_MONTH_DUE"  |"DAY_OF_WEEK_DUE"  |"YEAR_INVOICED"  |"MONTH_INVOICED"  |"DAY_OF_MONTH_INVOICED"  |"DAY_OF_WEEK_INVOICED"  |"UNPAID"  |"MaxCategoriaVehicul

In [112]:
df.show()

-------------------------------------------------------------------------------------------------
|"ImportePesos"  |"CantidadCuentasContrato"  |"CantidadVehiculos"  |"Score Equifax"  |"UNPAID"  |
-------------------------------------------------------------------------------------------------
|578.0           |1                          |4                    |696.0            |0         |
|2049.0          |1                          |1                    |881.0            |0         |
|2623.0          |1                          |3                    |517.0            |0         |
|5.0             |1                          |3                    |517.0            |0         |
|2209.0          |1                          |3                    |922.0            |0         |
|9.0             |1                          |3                    |922.0            |0         |
|471.0           |1                          |3                    |922.0            |0         |
|-9.0            |1 

In [111]:
normalized_df.show()

-------------------------------------------------------------------------------------------------------------------
|"""ImportePesos""_NORM"  |"""Score Equifax""_NORM"  |"CantidadCuentasContrato"  |"CantidadVehiculos"  |"UNPAID"  |
-------------------------------------------------------------------------------------------------------------------
|0.16118735786121743      |0.7805611222444889        |2                          |2                    |0         |
|0.1611614591879356       |0.24148296593186372       |2                          |4                    |0         |
|0.16116050686296415      |NULL                      |1                          |6                    |0         |
|0.16116001765493088      |NULL                      |1                          |6                    |0         |
|0.16116222235246752      |NULL                      |1                          |9                    |0         |
|0.1611623169326873       |NULL                      |1                 

In [104]:
# Group by the 'UNPAID' column and count each unique value
value_counts_df = normalized_df.group_by("UNPAID").agg(count("*").alias("count"))

# Collect the result to display or use further
value_counts_result = value_counts_df.collect()

# Display the result
for row in value_counts_result:
    print(row)

Row(UNPAID=1, COUNT=47573666)
Row(UNPAID=0, COUNT=435131741)


In [105]:
# Define the name of the table you want to create or overwrite in Snowflake
table_name = 'table_with_split'

# Write the DataFrame to a new table in Snowflake
normalized_df.write.mode('overwrite').save_as_table(table_name)

In [106]:
df_model=new_session.sql('select * from TABLE_WITH_SPLIT ')
print(df_model.columns)

['"""ImportePesos""_NORM"', '"""Score Equifax""_NORM"', '"CantidadCuentasContrato"', '"CantidadVehiculos"', 'UNPAID']


In [107]:
# Split the data into train and test sets
train_df, test_df = df_model.random_split(weights=[0.8, 0.2], seed=0)

In [108]:
all_columns = df_model.columns
filtered_columns = [col for col in all_columns if col != "UNPAID"]

input_columns=filtered_columns
label_columns="UNPAID"
output_columns="PREDICTED_DEFAULT"

In [109]:
from snowflake.ml.modeling.xgboost import XGBClassifier
# Define the XGBRegressor
regressor = XGBClassifier(
    input_cols=input_columns,
    label_cols=label_columns,
    output_cols=output_columns
)


In [110]:
# Train
regressor.fit(train_df)

KeyboardInterrupt: 

In [77]:
# Predict
result = regressor.predict(test_df)

In [79]:
result.show(100)

-----------------------------------------------------------------------------------------------------------------------------------------
|"""ImportePesos""_NORM"  |"""Score Equifax""_NORM"  |"CantidadCuentasContrato"  |"CantidadVehiculos"  |"UNPAID"  |"PREDICTED_DEFAULT"  |
-----------------------------------------------------------------------------------------------------------------------------------------
|0.0009707995542199661    |0.9999999999999999        |2                          |1                    |0         |0                    |
|0.0010739035641550221    |0.9358717434869739        |1                          |2                    |0         |0                    |
|0.001329925880960049     |0.4418837675350701        |1                          |7                    |0         |0                    |
|0.006743465638561363     |0.7595190380761523        |1                          |5                    |0         |0                    |
|0.0020400692302880194    |0.75951

In [66]:
from snowflake.ml.modeling.metrics import precision_score

# Assuming 'result' is your DataFrame containing the predictions and actual labels
# You need to specify the column names as strings and provide them as keyword arguments
precision = precision_score(
    df=result,
    y_true_col_names='"UNPAID"',  # Column name for the actual labels
    y_pred_col_names='"PREDICTED_DEFAULT"',  # Column name for the predicted labels
    average='binary',  # Assuming binary classification, adjust as needed
    pos_label=1  # Assuming 1 is the positive class, adjust as needed
)

# Print or use the precision value
print(f"Precision: {precision}")

Precision: 0.0


/usr/local/lib/python3.10/dist-packages/snowflake/ml/modeling/metrics/classification.py:1244: UndefinedMetricWarning: precision_score: division by zero: score value will be 0.
  return _binary_precision_score(


In [ ]:
from snowflake.ml.modeling.metrics import recall_score

# Assuming 'result' is your DataFrame containing the predictions and actual labels
# You need to specify the column names as strings and provide them as keyword arguments
recall = recall_score(
    df=result,
    y_true_col_names='"UNPAID"',  # Column name for the actual labels
    y_pred_col_names='"PREDICTED_DEFAULT"',  # Column name for the predicted labels
    average='binary',  # Assuming binary classification, adjust as needed
    pos_label=1  # Assuming 1 is the positive class, adjust as needed
)

# Print or use the precision value
print(f"Recall: {recall}")

In [16]:
def preprocessing(df):
  df = df.with_column("YEAR_DUE", to_varchar(year(col('"FechaVencimiento"'))))\
                            .with_column("MONTH_DUE", to_varchar(month(col('"FechaVencimiento"'))))\
                            .with_column("DAY_OF_MONTH_DUE", to_varchar(dayofmonth(col('"FechaVencimiento"'))))\
                            .with_column("DAY_OF_WEEK_DUE", to_varchar(dayofweek(col('"FechaVencimiento"'))))\

  # df = df.with_column("YEAR_PAID", to_varchar(year(col('"FechaCompensacion"'))))\
  #                           .with_column("MONTH_PAID", to_varchar(month(col('"FechaCompensacion"'))))\
  #                           .with_column("DAY_OF_MONTH_PAID", to_varchar(dayofmonth(col('"FechaCompensacion"'))))\
  #                           .with_column("DAY_OF_WEEK_DUE", to_varchar(dayofweek(col('"FechaCompensacion"'))))\

  df = df.with_column("YEAR_INVOICED", to_varchar(year(col('"FechaEmision"'))))\
                            .with_column("MONTH_INVOICED", to_varchar(month(col('"FechaEmision"'))))\
                            .with_column("DAY_OF_MONTH_INVOICED", to_varchar(dayofmonth(col('"FechaEmision"'))))\
                            .with_column("DAY_OF_WEEK_INVOICED", to_varchar(dayofweek(col('"FechaEmision"'))))\

  df = df.filter(col("RUT").is_not_null())
  df = df.withColumn('Unpaid', when(col('"StatusCompensacion"') == 'Impaga', 1).otherwise(0))

  #df = df.filter(col('YEAR_INVOICED')!=2024)


#   aggregated_df = df.group_by(col('"BusinessPartnerId"')).agg(
#     min(col("YEAR_INVOICED")).alias("first_year_joined"),
#     max(col("YEAR_INVOICED")).alias("last_year_present"),
#     (max(col("YEAR_INVOICED")) - min(col("YEAR_INVOICED"))).alias("customer_lifespan"),
#     (sum_(col('Unpaid')) / count(col('Unpaid')) * 100).alias("default_rate_percent"),  # Calculating default rate as a percentage
#     median(col('"ImportePesos"')).alias("median_importe_pesos"),
#     median(col('"DiasCompensacion"')).alias("median_dias_compensacion")
#     # avg(col('"Score Equifax"')).alias("average_score_equifax")
# )

#   print(aggregated_df)


    #dropping unique identifiers
  columns_to_drop = ['"NumeroDocumento"', '"NumeroFolio"', '"BusinessPartnerId"', '"RUT"','"CuentaContrato"','"StatusCompensacionTmp"',
                      '"FechaRegistro"','"ClaveReconciliacion"','"Cabecera_FechaHoraCreacion"','"Detalle_FechaHoraCreacion"','"Direccion"','"Nombre"','FECHAEXTRACCION','"DocumentoCompensacion"','"PosicionDocumento"',
                     ]
  df = df.drop(*columns_to_drop)

  #over 70% missing values
  columns_to_drop2=['"Cabecera_FechaHoraUltimaModificacion"', '"Detalle_FechaHoraUltimaModificacion"']
  df = df.drop(*columns_to_drop2)

  #redundant information- TipoOperacion contains the same information
  columns_to_drop3=['"OperacionPrincipal"','"OperacionParcial"','"TramoMora"']
  df = df.drop(*columns_to_drop3)


  df = df.with_column('"MaxCategoriaVehiculo"', col('"MaxCategoriaVehiculo"').astype("string"))
  df = df.drop('"StatusCompensacion"')
  df = df.drop('"DiasCompensacion"')

  schema = df.schema

  numerical_columns = []
  categorical_columns = []
  datetime_columns=[]

  # Iterate over the schema and check the data type of each column
  for field in schema.fields:
      #print(field.datatype)
      if isinstance(field.datatype, (IntegerType, FloatType, DoubleType, DecimalType, LongType)):
          numerical_columns.append(field.name)
      elif isinstance(field.datatype, StringType):
          categorical_columns.append(field.name)
      elif isinstance(field.datatype, (TimestampType, DateType)):
          datetime_columns.append(field.name)


  df = df.drop(*datetime_columns)


  # for column in categorical_columns:
  #   df = df.with_column(column, col(column).cast("category"))

  selected_columns = ['"ImportePesos"', '"Score Equifax"']
  numerical_columns_x = [col for col in numerical_columns if col in selected_columns]

  categorical_columns_ohe = [(col.strip('"') + '_OHE') for col in categorical_columns]


  num_columns_nor = [col + "_NORM" for col in numerical_columns_x]
  print(categorical_columns_ohe)

  #Encode categoricals to numeric columns
  snowml_ohe = snowml.OneHotEncoder(input_cols=categorical_columns, output_cols=categorical_columns_ohe)
  transformed_df = snowml_ohe.fit(df).transform(df)
  # print(transformed_df.show())

  #Scale numerical columns to prevent bias
  snowml_mms = snowml.MinMaxScaler(input_cols=numerical_columns_x, output_cols=num_columns_nor)
  normalized_df = snowml_mms.fit(transformed_df).transform(transformed_df)

  normalized_df=  normalized_df.drop(*selected_columns)

    #Add the outlier removal part if required
    #try adding year first joined to see if it changes the accuracy of the model
    #Check multicollinearity

  return normalized_df, numerical_columns, categorical_columns
  #return df

In [18]:
df_trial=df_con_equifax_v.limit(100)
df_trial.show(100)



-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"RUT"       |"FECHAEXTRACCION"  |"NumeroFolio"  |"FechaEmision"  |"FechaVencimiento"  |"FechaRegistro"  |"BusinessPartnerId"  |

In [19]:
df_processed, agg_df, num_cols, cat_cols = preprocessing(df_trial)

['COMUNA_OHE', 'CIUDAD_OHE', 'REGION_OHE', 'TramoMoraCliente_OHE', 'Inhabilitado_OHE', 'Concesionaria_OHE', 'TipoCliente_OHE', 'TipoOperacion_OHE', 'TipoDocumento_OHE', 'TipoProducto_OHE', 'YEAR_DUE_OHE', 'MONTH_DUE_OHE', 'DAY_OF_MONTH_DUE_OHE', 'DAY_OF_WEEK_DUE_OHE', 'YEAR_INVOICED_OHE', 'MONTH_INVOICED_OHE', 'DAY_OF_MONTH_INVOICED_OHE', 'DAY_OF_WEEK_INVOICED_OHE', 'MaxCategoriaVehiculo_OHE']


ValueError: (2112) Found unknown categories during transform:
              COLUMN_NAME      UNKNOWN_VALUE
0   DAY_OF_MONTH_INVOICED                 13
1                  COMUNA          LOS ANDES
2                YEAR_DUE               2020
3               MONTH_DUE                 10
4   DAY_OF_MONTH_INVOICED                  6
5                YEAR_DUE               2022
6                  COMUNA              ÑUÑOA
7         "TipoDocumento"  Boleta Infractora
8        DAY_OF_MONTH_DUE                  1
9        DAY_OF_MONTH_DUE                  3
10                 COMUNA              LAMPA
11                 CIUDAD          MELIPILLA
12                 COMUNA           CURACAVI
13          YEAR_INVOICED               2016
14              MONTH_DUE                 11
15                 CIUDAD             LIMARI
16       DAY_OF_MONTH_DUE                  7
17                 COMUNA           RECOLETA
18       DAY_OF_MONTH_DUE                 23
19   DAY_OF_WEEK_INVOICED             

In [ ]:
df_processed.show(5)

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"""ImportePesos""_NORM"  |"""Score Equifax""_NORM"  |"OperacionPrincipal"  |"OperacionParcial"  |"COMUNA"     |"CIUDAD"    |"REGION"  |"CantidadCuentasContrato"  |"CantidadVehiculos"  |"TramoMoraCliente"  |"TramoMora"  |"Inhabilitado"  |"Concesionaria"  |"TipoCliente"  |"TipoOperacion"   |"TipoDocumento"  |"TipoProducto"  |"YEAR_DUE"  |"MONTH_DUE"  |"DAY_OF_MONTH_DUE"  |"DAY_OF_WEEK_DUE"  |"YEAR_INVOICED"  |"MONTH_INVOICED"  |"DAY_OF_MONTH_INVOICED"  |"DAY_OF_WEEK_INVOICED"  

In [ ]:
agg_df.show(5)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"BusinessPartnerId"  |"FIRST_YEAR_JOINED"  |"LAST_YEAR_PRESENT"  |"CUSTOMER_LIFESPAN"  |"DEFAULT_RATE_PERCENT"  |"MEDIAN_IMPORTE_PESOS"  |"MEDIAN_DIAS_COMPENSACION"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0012313468           |2014                 |2021                 |7.0                  |0.000000                |210.0                   |3.0                         |
|0014413508           |2024                 |2024                 |0.0                  |60.000000               |801.0                   |-15.0                       |
|0010576292           |2008                 |2023                 |15.0                 |0.000000                |876.5                   |-18.0           

In [ ]:
num_cols

['"ImportePesos"',
 '"CantidadCuentasContrato"',
 '"CantidadVehiculos"',
 '"Score Equifax"',
 'UNPAID']

In [ ]:
cat_cols

['"OperacionPrincipal"',
 '"OperacionParcial"',
 'COMUNA',
 'CIUDAD',
 'REGION',
 '"TramoMoraCliente"',
 '"TramoMora"',
 '"Inhabilitado"',
 '"Concesionaria"',
 '"TipoCliente"',
 '"TipoOperacion"',
 '"TipoDocumento"',
 '"TipoProducto"',
 'YEAR_DUE',
 'MONTH_DUE',
 'DAY_OF_MONTH_DUE',
 'DAY_OF_WEEK_DUE',
 'YEAR_INVOICED',
 'MONTH_INVOICED',
 'DAY_OF_MONTH_INVOICED',
 'DAY_OF_WEEK_INVOICED',
 '"MaxCategoriaVehiculo"']

In [ ]:
# Define the name of the table you want to create or overwrite in Snowflake
table_name = 'table_with_split'

# Write the DataFrame to a new table in Snowflake
df_processed.write.mode('overwrite').save_as_table(table_name)

In [ ]:
df_model=new_session.sql('select * from TABLE_WITH_SPLIT ')
print(df_model.columns)

['"""ImportePesos""_NORM"', '"""Score Equifax""_NORM"', '"OperacionPrincipal"', '"OperacionParcial"', 'COMUNA', 'CIUDAD', 'REGION', '"CantidadCuentasContrato"', '"CantidadVehiculos"', '"TramoMoraCliente"', '"TramoMora"', '"Inhabilitado"', '"Concesionaria"', '"TipoCliente"', '"TipoOperacion"', '"TipoDocumento"', '"TipoProducto"', 'YEAR_PAID', 'MONTH_PAID', 'DAY_OF_MONTH_PAID', 'DAY_OF_WEEK_DUE', 'YEAR_INVOICED', 'MONTH_INVOICED', 'DAY_OF_MONTH_INVOICED', 'DAY_OF_WEEK_INVOICED', 'UNPAID', '"MaxCategoriaVehiculo"']


In [ ]:
df_model_trial=df_model.limit(100000)

In [ ]:
# Split the data into train and test sets
train_df, test_df = df_model_trial.random_split(weights=[0.8, 0.2], seed=0)

In [ ]:
train_df_X=train_df.drop('UNPAID')
train_df_Y=train_df.select(col('UNPAID'))

In [ ]:
train_df_X.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"""ImportePesos""_NORM"  |"""Score Equifax""_NORM"  |"OperacionPrincipal"  |"OperacionParcial"  |"COMUNA"    |"CIUDAD"   |"REGION"  |"CantidadCuentasContrato"  |"CantidadVehiculos"  |"TramoMoraCliente"  |"TramoMora"  |"Inhabilitado"  |"Concesionaria"  |"TipoCliente"  |"TipoOperacion"  |"TipoDocumento"      |"TipoProducto"  |"YEAR_PAID"  |"MONTH_PAID"  |"DAY_OF_MONTH_PAID"  |"DAY_OF_WEEK_DUE"  |"YEAR_INVOICED"  |"MONTH_INVOICED"  |"DAY_OF_MONTH_INVOICED"  |"DAY_OF_WEEK_INVOICED"  |"M

In [ ]:
train_df_Y.show()

------------
|"UNPAID"  |
------------
|1         |
|1         |
|1         |
|0         |
|0         |
|0         |
|0         |
|0         |
|0         |
|0         |
------------



In [ ]:
train_df.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"""ImportePesos""_NORM"  |"""Score Equifax""_NORM"  |"OperacionPrincipal"  |"OperacionParcial"  |"COMUNA"     |"CIUDAD"    |"REGION"  |"CantidadCuentasContrato"  |"CantidadVehiculos"  |"TramoMoraCliente"  |"TramoMora"  |"Inhabilitado"  |"Concesionaria"  |"TipoCliente"  |"TipoOperacion"   |"TipoDocumento"  |"TipoProducto"  |"YEAR_PAID"  |"MONTH_PAID"  |"DAY_OF_MONTH_PAID"  |"DAY_OF_WEEK_DUE"  |"YEAR_INVOICED"  |"MONTH_INVOICED"  |"DAY_OF_MONTH_INVOICED"  |"DAY_OF_WEEK_INVOI

In [ ]:
test_df.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"""ImportePesos""_NORM"  |"""Score Equifax""_NORM"  |"OperacionPrincipal"  |"OperacionParcial"  |"COMUNA"       |"CIUDAD"    |"REGION"  |"CantidadCuentasContrato"  |"CantidadVehiculos"  |"TramoMoraCliente"  |"TramoMora"  |"Inhabilitado"  |"Concesionaria"  |"TipoCliente"  |"TipoOperacion"   |"TipoDocumento"      |"TipoProducto"  |"YEAR_PAID"  |"MONTH_PAID"  |"DAY_OF_MONTH_PAID"  |"DAY_OF_WEEK_DUE"  |"YEAR_INVOICED"  |"MONTH_INVOICED"  |"DAY_OF_MONTH_INVOICED"  |"DAY_O

In [ ]:
all_columns = df_model_trial.columns
filtered_columns = [col for col in all_columns if col != "UNPAID"]

input_columns=filtered_columns
label_columns="UNPAID"
output_columns="PREDICTED_DEFAULT"

In [ ]:
input_columns

['"""ImportePesos""_NORM"',
 '"""Score Equifax""_NORM"',
 '"OperacionPrincipal"',
 '"OperacionParcial"',
 'COMUNA',
 'CIUDAD',
 'REGION',
 '"CantidadCuentasContrato"',
 '"CantidadVehiculos"',
 '"TramoMoraCliente"',
 '"TramoMora"',
 '"Inhabilitado"',
 '"Concesionaria"',
 '"TipoCliente"',
 '"TipoOperacion"',
 '"TipoDocumento"',
 '"TipoProducto"',
 'YEAR_PAID',
 'MONTH_PAID',
 'DAY_OF_MONTH_PAID',
 'DAY_OF_WEEK_DUE',
 'YEAR_INVOICED',
 'MONTH_INVOICED',
 'DAY_OF_MONTH_INVOICED',
 'DAY_OF_WEEK_INVOICED',
 '"MaxCategoriaVehiculo"']

In [ ]:
from snowflake.ml.modeling.xgboost import XGBRegressor
# Define the XGBRegressor
regressor = XGBRegressor(
    input_cols=input_columns,
    label_cols=label_columns,
    output_cols=output_columns
)


In [ ]:
# Train
regressor.fit(train_df)

SnowparkSQLException: (1300) (1304): 01b40b5d-0105-1df9-0004-c9db0013f1b6: 100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "/home/udf/80337836/udf_py_123455646.zip/udf_py_123455646.py", line 76, in compute
    return func(session,arg1,arg2,arg3,arg4,arg5,arg6,arg7)
  File "/usr/local/lib/python3.10/dist-packages/snowflake/ml/modeling/_internal/snowpark_implementations/snowpark_trainer.py", line 233, in fit_wrapper_function
  File "/usr/lib/python_udf/bde3624bc62adcfa9b4bcca39aaadf3cae31d009ca3c98b5429fc3575c4456fd/lib/python3.10/site-packages/xgboost/core.py", line 620, in inner_f
    return func(**kwargs)
  File "/usr/lib/python_udf/bde3624bc62adcfa9b4bcca39aaadf3cae31d009ca3c98b5429fc3575c4456fd/lib/python3.10/site-packages/xgboost/sklearn.py", line 988, in fit
    train_dmatrix, evals = _wrap_evaluation_matrices(
  File "/usr/lib/python_udf/bde3624bc62adcfa9b4bcca39aaadf3cae31d009ca3c98b5429fc3575c4456fd/lib/python3.10/site-packages/xgboost/sklear

In [ ]:
# Predict
result = regressor.predict(test_df)

In [ ]:
from snowflake.ml.modeling.metrics.correlation import correlation

In [ ]:
corr_df = correlation(df=df_model)
corr_df

,"""""""ImportePesos""""_NORM""","""""""Score Equifax""""_NORM""","""CantidadCuentasContrato""","""CantidadVehiculos""",UNPAID
"""""""ImportePesos""""_NORM""",1.000000,NaN,0.011998,NaN,0.020792
"""""""Score Equifax""""_NORM""",NaN,NaN,NaN,NaN,NaN
"""CantidadCuentasContrato""",0.011998,NaN,1.000000,NaN,-0.023315
"""CantidadVehiculos""",NaN,NaN,NaN,NaN,NaN
UNPAID,0.020792,NaN,-0.023315,NaN,1.000000
